In [1]:
from datetime import datetime as dt, time
from datetime import timedelta
import mongo_queries

In [13]:
db = mongo_queries.db
col= db['baseline']
run = db['running']
leg = db['legacy']
late = db['latest times']
#merge = db['running_merge']
merge = db['running_merge2'] #for smaller queries
lm = db['latest_merged']
current = db['current_updaters']
profs = db['profiles']

In [2]:
curr = dt.utcnow()
y = curr.year
m = curr.month

start = dt(y,m,1)
if m==12:
    end = dt(y+1,1,1) - timedelta(days=1)
else:
    end = dt(y,m+1,1) - timedelta(days=1)


In [5]:
start

datetime.datetime(2021, 11, 1, 0, 0)

In [6]:
end

datetime.datetime(2021, 11, 30, 0, 0)

In [7]:
curr = dt(2021,9,15)

In [4]:
mongo_queries.queries_this_month_base(curr)

7703

In [8]:
curr

datetime.datetime(2021, 9, 15, 0, 0)

In [14]:
def queries_this_month_base(curr=None):
    #how many queries were sent to google this month, if a date is supplied it will use that calendar month. If not, it will use today (UTC)
    if not curr:
        curr = dt.utcnow()
    y = curr.year
    m = curr.month

    start = dt(y,m,1)
    if m==12:
        end = dt(y+1,1,1) - timedelta(days=1) #if its in December, then you go to the day before Jan 1st.
    else:
        end = dt(y,m+1,1) - timedelta(days=1) #otherwise go to the next month, and go back 1 day. This handles the months of varying length

    res = col.aggregate([
    {
        '$unwind': {
            'path': '$wait_times', 
            'includeArrayIndex': 'string', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$project': {
            'utc': '$wait_times.utc'
        }
    }, {
        '$match': {
            'utc': {
                '$gte': start, 
                '$lt': end
            }
        }
    }, {
        '$count': 'utc'
    }
])

In [15]:
queries_this_month_base()